In [20]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 7: Recursive Circus ---</h2><p>Wandering further through the circuits of the computer, you come upon a tower of <span title="Turtles, all the way down.">programs</span> that have gotten themselves into a bit of trouble.  A recursive algorithm has gotten out of hand, and now they're balanced precariously in a large tower.</p>
<p>One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding <em>their</em> own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.</p>
<p>You offer to help, but first you need to understand the structure of these towers.  You ask each program to yell out their <em>name</em>, their <em>weight</em>, and (if they're holding a disc) the <em>names of the programs immediately above them</em> balancing on that disc. You write this information down (your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion; by the time you're done, you're not sure which program gave which information.</p>
<p>For example, if your list is the following:</p>
<pre><code>pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -&gt; ktlj, cntj, xhth
qoyq (66)
padx (45) -&gt; pbga, havc, qoyq
tknk (41) -&gt; ugml, padx, fwft
jptl (61)
ugml (68) -&gt; gyxo, ebii, jptl
gyxo (61)
cntj (57)
</code></pre>
<p>...then you would be able to recreate the structure of the towers that looks like this:</p>
<pre><code>                gyxo
              /     
         ugml - ebii
       /      \     
      |         jptl
      |        
      |         pbga
     /        /
tknk --- padx - havc
     \        \
      |         qoyq
      |             
      |         ktlj
       \      /     
         fwft - cntj
              \     
                xhth
</code></pre>
<p>In this example, <code>tknk</code> is at the bottom of the tower (the <em>bottom program</em>), and is holding up <code>ugml</code>, <code>padx</code>, and <code>fwft</code>.  Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much larger.)</p>
<p>Before you're ready to help them, you need to make sure your information is correct.  <em>What is the name of the bottom program?</em></p>
</article>


In [21]:
from more_itertools import one


example = """
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)

"""


def get_bottom(sketch: str) -> str:
    nodes = set()
    not_root = set()
    for line in sketch.strip().splitlines():
        node, *rest = re.split(r",*\s", line)

        if len(rest) > 1:
            nodes.add(node)
        not_root |= set(rest[2:])

    return one(nodes - not_root)


assert get_bottom(example) == "tknk"

In [22]:
with open("../input/day7.txt") as f:
    puzzle = f.read()

print(f"Part I: {get_bottom(puzzle)}")

Part I: vvsvez


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>vvsvez</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The programs explain the situation: they can't get down.  Rather, they <em>could</em> get down, if they weren't expending all of their energy trying to keep the tower balanced. Apparently, one program has the <em>wrong weight</em>, and until it's fixed, they're stuck here.</p>
<p>For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.</p>
<p>In the example above, this means that for <code>ugml</code>'s disc to be balanced, <code>gyxo</code>, <code>ebii</code>, and <code>jptl</code> must all have the same weight, and they do: <code>61</code>.</p>
<p>However, for <code>tknk</code> to be balanced, each of the programs standing on its disc <em>and all programs above it</em> must each match. This means that the following sums must all be the same:</p>
<ul>
<li><code>ugml</code> + (<code>gyxo</code> + <code>ebii</code> + <code>jptl</code>) = 68 + (61 + 61 + 61) = 251</li>
<li><code>padx</code> + (<code>pbga</code> + <code>havc</code> + <code>qoyq</code>) = 45 + (66 + 66 + 66) = 243</li>
<li><code>fwft</code> + (<code>ktlj</code> + <code>cntj</code> + <code>xhth</code>) = 72 + (57 + 57 + 57) = 243</li>
</ul>
<p>As you can see, <code>tknk</code>'s disc is unbalanced: <code>ugml</code>'s stack is heavier than the other two. Even though the nodes above <code>ugml</code> are balanced, <code>ugml</code> itself is too heavy: it needs to be <code>8</code> units lighter for its stack to weigh <code>243</code> and keep the towers balanced.  If this change were made, its weight would be <code>60</code>.</p>
<p>Given that exactly one program is the wrong weight, <em>what would its weight need to be</em> to balance the entire tower?</p>
</article>

</main>


In [23]:
from collections import Counter
from more_itertools import all_equal


def balance_tower(sketch: str) -> int:
    # Doesn't work if the unbalnced node has a parent that has two childs
    def dfs(node: str) -> tuple[int, int]:
        children = [dfs(c) for c in graph[node][1]]
        values = Counter(wt for _, wt in children)

        if len(values) > 1:
            nonlocal balancing_weight
            desired_weight = one(w for w, c in values.items() if c > 1)
            current_weight = one(w for w, c in values.items() if c == 1)
            balancing_weight = one(wc for wc, wt in children if wt != desired_weight)
            balancing_weight += desired_weight - current_weight
            return graph[node][0], desired_weight * len(children) + graph[node][0]

        return graph[node][0], sum((ws for _, ws in children), start=graph[node][0])

    root = get_bottom(sketch)
    graph = create_graph(sketch)
    balancing_weight = -1
    dfs(root)
    return balancing_weight


def create_graph(sketch):
    graph = {}

    for line in sketch.strip().splitlines():
        node, *rest = re.split(r",*\s", line)
        graph[node] = int(rest[0][1:-1]), rest[2:]
    return graph


balance_tower(example)

60

In [24]:
print(f"Part II: {balance_tower(puzzle)}")
# not 8

Part II: 362


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>362</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


<link href="style.css" rel="stylesheet"></link>
